In [ ]:
# 반드시 런타임 다시시작을 하셔야 합니다.
!pip install -q tensorflow-data-validation

In [ ]:
import sys, os
import tempfile, urllib, zipfile
import tensorflow_data_validation as tfdv

print('TFDV version: {}'.format(tfdv.version.__version__))

TFDV version: 0.27.0


In [ ]:
[methods for methods in dir(tfdv) if "generate" in methods]

['generate_statistics_from_csv',
 'generate_statistics_from_dataframe',
 'generate_statistics_from_tfrecord']

# Static

In [ ]:
train_stats = tfdv.generate_statistics_from_csv(data_location='/content/sample_data/california_housing_train.csv')

In [ ]:
tfdv.visualize_statistics(train_stats)

In [ ]:
eval_stats = tfdv.generate_statistics_from_csv(data_location='/content/sample_data/california_housing_test.csv')

tfdv.visualize_statistics(lhs_statistics=eval_stats, rhs_statistics=train_stats, lhs_name='EVAL_DATASET', rhs_name='TRAIN_DATASET')

# Schema

In [ ]:
schema = tfdv.infer_schema(statistics = train_stats)
schema

feature {
  name: "longitude"
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "latitude"
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "housing_median_age"
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "total_rooms"
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "total_bedrooms"
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "population"
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "households"
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape

In [ ]:
tfdv.display_schema(schema=schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'longitude',FLOAT,required,,-
'latitude',FLOAT,required,,-
'housing_median_age',FLOAT,required,,-
'total_rooms',FLOAT,required,,-
'total_bedrooms',FLOAT,required,,-
'population',FLOAT,required,,-
'households',FLOAT,required,,-
'median_income',FLOAT,required,,-
'median_house_value',FLOAT,required,,-


In [ ]:
tfdv.get_feature(schema, 'population').presence.min_fraction = 0.8

In [ ]:
tfdv.get_feature(schema, 'population')

name: "population"
type: FLOAT
presence {
  min_fraction: 0.8
  min_count: 1
}
shape {
  dim {
    size: 1
  }
}

In [ ]:
anomalies = tfdv.validate_statistics(statistics=eval_stats, schema=schema)
tfdv.display_anomalies(anomalies)

# Drift & Skew

In [ ]:
payment_type = tfdv.get_feature(schema, 'payment_type')
payment_type.skew_comparator.infinity_norm.threshold = 0.01

In [ ]:
company=tfdv.get_feature(schema, 'company')
company.drift_comparator.infinity_norm.threshold = 0.001

skew_anomalies = tfdv.validate_statistics(train_stats, schema,
                                          previous_statistics=eval_stats,
                                          serving_statistics=serving_stats)

tfdv.display_anomalies(skew_anomalies)